In [ ]:
#Load The Data
import pickle
training_file   = 'traffic-signs-data/train.p'
validation_file = 'traffic-signs-data/valid.p'
test_file       = 'traffic-signs-data/test.p'

with open(training_file , mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(test_file, mode='rb') as f:
    test  = pickle.load(f)
    
X_train,Y_train   = train['features'], train['labels']
X_valid,Y_valid   = valid['features'], valid['labels']
X_test, Y_test    = test['features'] , test['labels']



In [ ]:
import numpy as np
# Summary to see what is available in the data
n_train   = X_train.shape[0]
n_valid   = X_valid.shape[0]
n_test    = X_test.shape[0]
n_images  = X_train.shape[1:]
n_classes = len(np.unique(Y_train))

print("Number of training examples =", n_train)
print("Number of validation examples",n_valid)
print("Number of testing examples =", n_test)
print("Image data shape =", n_images)
print("Number of classes =", n_classes)

In [ ]:
#Data Visualization
import matplotlib.pyplot as plt
import csv
labels = []
with open('signnames.csv','r') as f:
    read_csv = csv.reader(f, delimiter=',')
    next(read_csv)
    
    for row in read_csv:
        labels += [row[1]]
        
plt.figure(figsize=(15,30))
num_classes = []
for i in range(0, n_classes):
    plt.subplot(15,4,i+1)
    select = X_train[Y_train == i]
    plt.imshow(select[0,:,:,:])
    plt.title(labels[i])
    plt.axis('off')
    num_classes.append(len(select))
plt.show()

In [ ]:
#Preprocessing The Images
from sklearn.utils import shuffle
X_train, Y_train = shuffle(X_train,Y_train)

In [ ]:
#Preprocessing The Images
def preprocessing_images(image_set):
    image_shape       = X_train.shape[1:]
    new_image         = image_shape[0:2] + (1,)
    prepare_image_set = np.empty(shape=(len(image_set),) + new_image , dtype=int) 
    for i in range(0 , len(image_set)):
        normalize_img         = cv2.normalize(image_set[i] , np.zeros(new_image[0:2]), 0, 255, cv2.NORM_MINMAX)
        gray_image            = cv2.cvtColor(normalize_img, cv2.COLOR_RGB2GRAY)
        prepare_image_set[i]  = np.reshape(gray_image, new_image)
    return prepare_image_set

In [ ]:
import cv2
X_train_pre  = preprocessing_images(X_train)
X_valid_pre  = preprocessing_images(X_valid)
X_test_pre   = preprocessing_images(X_test)

In [ ]:
image_shape_prep = X_train_pre[0].shape


In [ ]:
#Model Architecture
import tensorflow as tf
from tensorflow.contrib.layers import flatten
def leNet(x, channels, classes, keep_prob, mu=0, sigma=0.01):
    #Layer 1
    conv1_W  = tf.Variable(tf.truncated_normal(shape=(5,5,channels,6), mean = mu, stddev = sigma))
    conv1_b  = tf.Variable(tf.zeros(6))
    conv1    = tf.nn.conv2d(x, conv1_W, strides=[1,1,1,1],padding='VALID') + conv1_b
    #Layer 1 Activattion
    conv1    = tf.nn.relu(conv1)
    #Layer 1 Pooling
    conv1    = tf.nn.max_pool(conv1 , ksize=[1,2,2,1], strides= [1,2,2,1], padding='VALID')
    
    #Layer 2
    conv2_W  = tf.Variable(tf.truncated_normal(shape=(5,5,6,16),mean=mu, stddev = sigma))
    conv2_b  = tf.Variable(tf.zeros(16))
    conv2    = tf.nn.conv2d(conv1, conv2_W, strides=[1,1,1,1], padding='VALID') + conv2_b
    #Layer 2 Activation
    conv2    = tf.nn.relu(conv2)
    #Layer 2 Pooling
    conv2    = tf.nn.max_pool(conv2, ksize=[1,2,2,1] , strides=[1,2,2,1], padding='VALID')
    
    #Flatten And Dropout
    fc0      = flatten(conv2)
    fc0      = tf.nn.dopout(fc0, keep_prob=keep_prob)
    #Fully Connected Layer 1
    fc1_W      = tf.Variable(tf.truncated_normal(shape=(400,120),mean=mu, stddev=sigma))
    fc1_b      = tf.Variable(tf.zeros(120))
    fc1        = tf.matmul(fc0 , fc1_W) + fc1_b
    
    fc1        = tf.nn.relu(fc1)
    #Fully Connected Layer 2
    fc2_W      = tf.Variable(tf.truncated_normal(shape=(120,84),mean=mu, stddev=sigma))
    fc2_b      = tf.Variable(tf.zeros(84))
    fc2        = tf.matmul(fc1 , fc2_W) + fc2_b
    fc2        = tf.matmul(fc2)
    #Fully Connected Layer 3
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, classes), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(classes))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [ ]:
x = tf.placeholder(tf.float32, (None,) + image_shape_prep)
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)
RATE = 0.0008
EPOCHS = 30
BATCH_SIZE = 128
KEEP_PROB = 0.7
STDDEV = 0.01

keep_prob = tf.placeholder(tf.float32)
logits = leNet(x, image_shape_prep[-1], n_classes, keep_prob, sigma=STDDEV)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = RATE)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
#Evaluation

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
#Training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train_prep)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train_prep, y_train_prep = shuffle(X_train_prep, y_train_prep)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_prep[offset:end], y_train_prep[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: KEEP_PROB})
            
        train_accuracy = evaluate(X_train_prep, y_train_prep)    
        validation_accuracy = evaluate(X_valid_prep, y_valid_prep)
        print("EPOCH {} ...".format(i+1))
        print("Train Accuracy = {:.3f}".format(train_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './model.ckpt')
    print("Model saved")

In [ ]:
#Testing
with tf.Session() as sess:
    saver.restore(sess, './model.ckpt')

    test_accuracy = evaluate(X_test_prep, y_test_prep)
    print("Test Accuracy = {:.3f}".format(test_accuracy))